In [ ]:
import numpy as np
from os import listdir
from pickle import dump
from pickle import load
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint


import matplotlib.pyplot as plt
import pickle as pkl
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 1 图片预处理

In [ ]:
# import cv2

# '''----------------test-------------------'''
# test = cv2.imread( 'Flickr8k_Dataset\\Flicker8k_Dataset\\'+img_names[0])
# print('h,w,c = ',test.shape)
# test = cv2.resize(test,(224,224))
# print('h,w,c = ',test.shape)

In [ ]:
import cv2
def reshape(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))  
    return img

In [ ]:
import os
#os.listdir(filename)返回filename中所有文件的文件名列表
img_names = os.listdir('Flickr8k_Dataset\Flicker8k_Dataset')
img_num = len(img_names)
img_num

In [ ]:
# '''----------------test-------------------'''
# x = np.expand_dims(test, axis=0)  # 展开
# x = preprocess_input(x)       # 预处理到0～1
# out = vgg19.predict(x)

# print(out.shape)
# type(out)

In [ ]:
img_mapping = dict()  
for i in range(img_num):
    img_path =  'Flickr8k_Dataset\\Flicker8k_Dataset\\'+img_names[i]
    img = reshape(img_path)
    name = img_names[i]
    img_mapping[name] = img

In [ ]:
'''===============test==============='''
img_mapping[img_names[1]].shape

In [ ]:
import pickle

if not os.path.exists(os.path.abspath('.') + '\\img_mapping.npy'):    
    with open('img_mapping.pkl', 'wb') as df1:
        pickle.dump(img_mapping, df1, pickle.HIGHEST_PROTOCOL)
else:
    with open('img_mapping.pkl', 'rb') as df2:
        img_mapping = pickle.load(df2)

## 2 处理语句信息

In [ ]:
import string

# load .txt file as text
def load_doc(txt_filepath):
    # open the file as read only
    file = open(txt_filepath, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# extract descriptions for images
def load_text(text):
    imgs=[]
    words = []
    img_ids = []
    annots = []
    for line in text.split('\n'):
        
        tokens = line.split()
        if len(line) > 1:
            
            # take the first token as the image id, the rest as the description
            img_id, img_wd = tokens[0], tokens[1:]
            # remove filename from image id
            img_id = img_id.split('#')[0]
            
            #build the list of ids
            if img_id not in img_ids:
                img_ids.append(img_id)
                imgs.append(img_mapping[img_id])
                annots.append([])
            
            # build a set of words
            for word in img_wd:
                word.strip('.,\"')
                word.lower()
                if word not in words:
                    words.append(word)
                    
            # convert description tokens back to string        
            annot = ' '.join(img_wd)
            #build the list of ids
            annots[-1].append(annot)
            
    return img_ids,imgs, annots, words


In [ ]:
text_path = 'Flickr8k_text\Flickr8k.lemma.token.txt'
text = load_doc(text_path)
ids, imgs, annots, words =load_text(text)

In [ ]:
import random
from random import shuffle
#import itertools 
z = list(zip( imgs, annots))
random.shuffle(z)
imgs, annots = zip(*z)

In [ ]:
wd_len = len(words)
wd_len

In [ ]:
#annots = np.array(annots)
# annots.shape

In [ ]:
# [item for sublist in annots for item in sublist]

In [ ]:
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts([item for sublist in annots for item in sublist])
# print(tokenizer.word_index) 

In [ ]:
annots_=annots
annots_
k = 0
while k < 8091:
    p = 0
    while p < 5:
        a = annots_[k][p]
        a = T.text_to_word_sequence(a)
        a = tokenizer.texts_to_sequences(a)
        a = [item for sublist in a for item in sublist]
        #a = np.array(a)
        print(a)
        annots_[k][p]=a
        p += 1
    k += 1
#annots_ = np.array(annots_)
# annots_

In [ ]:
# annots_.shape

In [ ]:
# '''----------------test-------------------'''
# import keras.preprocessing.text as T
# from keras.preprocessing.text import Tokenizer
# tokenizer = Tokenizer(num_words=None)
# tokenizer.fit_on_texts(texts)
# text1='some thing to eat'
# text2='some thing to drink'
# texts=[text1,text2]

# print (T.text_to_word_sequence(text2) )

## 3 制做数据集

In [ ]:
from sklearn.model_selection import train_test_split
img_train, img_test, annot_train, annot_test = train_test_split(imgs,annots_,test_size=0.25, random_state=42)
img_test, img_val, annot_test, annot_val = train_test_split(img_test, annot_test, test_size=0.5, random_state=42)

## 4 搭建模型

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
from keras.utils import plot_model
# define the decoder model
def define_model(vocab_size, max_length):
    
    pretrained_cnn = VGG19(weights='imagenet', include_top=False)
    pretrained_cnn.layers.pop()
    # pretrained_cnn.trainable = True
    for layer in pretrained_cnn.layers[:]:
        layer.trainable = True
    
    # feature extractor model
    inputs1 = Input(shape=(224,224,3))
#     Encoder = VGG19(weights='imagenet', include_top=True)
#     Encoder.layers.pop()
    fe0 = pretrained_cnn(inputs1)
    fe1 = GlobalAveragePooling2D()(fe0)
    fe2 = Dropout(0.5)(fe1)
    fe3 = Dense(256, activation='relu')(fe2)

    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # decoder model
    decoder1 = add([fe3, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    
    return model

In [ ]:
model = define_model(vocab_size, 30)

In [ ]:
len(annot_train)

In [ ]:
#the below function loop forever with a while loop and within this, 
#loop over each image in the image directory. 
#For each image filename, we can load the image and 
#create all of the input-output sequence pairs from the image’s description.

#data generator, intended to be used in a call to model.fit_generator()
def data_generator(annots_train_r,img_train, max_length):
    while 1:
        for k in range(6068):
            #retrieve photo features
            img = img_train[k]
            annots_5 = annot_train[k]
            input_image, input_sequence, output_word = create_sequences( max_length, annots_5, img)
            yield [[input_image, input_sequence], output_word]

            
#we are calling the create_sequence() function to create 
#a batch worth of data for a single photo rather than an entire dataset. 
#This means that we must update the create_sequences() function 
#to delete the “iterate over all descriptions” for-loop.            
#Updated create sequence function for data_generator
def create_sequences(max_length, annots_5, img):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for annot in annots_5:
        # split one sequence into multiple X,y pairs
        for s in range(1, len(annot)):
            # split into input and output pair
            in_seq, out_seq = annot[:s], annot[s]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(img)
            X2.append(in_seq)
            y.append(out_seq)
    
    return np.array(X1), np.array(X2), np.array(y)

In [ ]:
# start_epoch = 0
# if ckpt_manager.latest_checkpoint:
#   start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

## 5 训练

In [ ]:
epochs = 20
steps = len(annot_train)
max_length = 30
# from time import time
# random.seed(time)
for i in range(epochs):
#    annot_train_r = []
#     for j in range(6068):
#         index = random.randint(0,4)
#         annot_train_r.append(annot_train[j][index])
    generator = data_generator(annot_train, img_train , max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    #model.fit(x={'inputs1': img_train, 'inputs2': annot_train_r}, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('model_' + str(i) + '.h5')